In [62]:
import pandas as pd
import numpy as np

In [63]:
df = pd.read_csv("gesture_dataset_cleaned.csv")

In [64]:
# Tạo danh sách đặc trưng
feature_cols = [col for col in df.columns if col not in ["frame_index", "label"]]
sequence_length = 10  # Số frame mỗi chuỗi

In [65]:
# Gom chuỗi liên tiếp theo nhãn
X = []
y = []

for label in df["label"].unique():
    group = df[df["label"] == label].sort_values("frame_index")
    data = group[feature_cols].values
    for i in range(len(data) - sequence_length + 1):
        X.append(data[i:i+sequence_length])  # Chuỗi frame
        y.append(label)

X = np.array(X)  # shape: (samples, sequence_length, features)
y = np.array(y)
print("✅ Dữ liệu:", X.shape, "Số nhãn:", len(np.unique(y)))

✅ Dữ liệu: (550, 10, 42) Số nhãn: 9


In [66]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_onehot = to_categorical(y_encoded)

print("🎯 Nhãn đã mã hóa:", le.classes_)

🎯 Nhãn đã mã hóa: ['a' 'b' 'c' 'd' 'e' 'q' 'r' 't' 'đ']


In [67]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y_onehot, test_size=0.2, random_state=42, stratify=y
)

In [68]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(64, input_shape=(sequence_length, X.shape[2])))
model.add(Dropout(0.3))
model.add(Dense(64, activation="relu"))
model.add(Dense(y_onehot.shape[1], activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()


d:\AI4LI\pytorch-openpose\openpose-env\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                   │ (None, 64)             │        27,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 9)              │           585 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,137 (125.54 KB)

 Trainable params: 32,137 (125.54 KB)

 Non-trainable params: 0 (0.00 B)

In [69]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=16
)

Epoch 1/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.2424 - loss: 2.1844 - val_accuracy: 0.7455 - val_loss: 1.5359
Epoch 2/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6630 - loss: 1.4458 - val_accuracy: 0.8091 - val_loss: 1.1210
Epoch 3/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7800 - loss: 1.0887 - val_accuracy: 0.9091 - val_loss: 0.7659
Epoch 4/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8692 - loss: 0.7607 - val_accuracy: 0.9364 - val_loss: 0.5329
Epoch 5/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8654 - loss: 0.6556 - val_accuracy: 0.9182 - val_loss: 0.4193
Epoch 6/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9261 - loss: 0.4267 - val_accuracy: 0.9636 - val_loss: 0.3267
Epoch 7/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8868 - loss: 0.4741 - val_accuracy: 0.9364 - val_loss: 0.2741
Epoch 8/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9345 - loss: 0.3198 - val_accuracy: 0.9727 - val_loss

In [70]:
import numpy as np
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
y_pred_label = le.inverse_transform(np.argmax(y_pred, axis=1))
y_true_label = le.inverse_transform(np.argmax(y_test, axis=1))

print(classification_report(y_true_label, y_pred_label))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
              precision    recall  f1-score   support

           a       1.00      1.00      1.00        14
           b       1.00      1.00      1.00         9
           c       1.00      1.00      1.00        10
           d       1.00      0.90      0.95        10
           e       1.00      1.00      1.00        12
           q       1.00      1.00      1.00        19
           r       0.94      1.00      0.97        17
           t       1.00      1.00      1.00        14
           đ       1.00      1.00      1.00         5

    accuracy                           0.99       110
   macro avg       0.99      0.99      0.99       110
weighted avg       0.99      0.99      0.99       110



In [71]:
import joblib
model.save("gesture_lstm_model.h5")
joblib.dump(le, "label_encoder.pkl")

['label_encoder.pkl']